In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestClassifier
from sklearn import model_selection, preprocessing
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import auc
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import collections
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data = pd.read_csv('linear_train.txt', header=None)
tst = pd.read_csv('linear_test.txt', header=None)
#data[0]=data[0].map(lambda x:x.lower())
#tst[0]=tst[0].map(lambda x:x.lower())

In [12]:
def prep(word):
    if(len(word)<=4):
        return word
    else:
        return word[-4:]

In [5]:
count_vectorizer = CountVectorizer(min_df=5, max_df=.9, max_features=None,ngram_range=(2, 5), analyzer='char_wb', lowercase=False, binary=True) 
X = count_vectorizer.fit_transform(data[0])

<101408x75077 sparse matrix of type '<class 'numpy.int64'>'
	with 2873474 stored elements in Compressed Sparse Row format>

In [14]:
y = data[1]
X = data.drop([0, 1], axis=1)

In [7]:
algo = LogisticRegression(penalty='l1', C=0.145, )
arr = cross_val_score(algo, X, y, cv=5, scoring='roc_auc')
arr

array([ 0.81470318,  0.77642342,  0.80657938,  0.78043748,  0.83847051])

In [8]:
model = LinearRegression()
arr = cross_val_score(model, X, y, cv=5, scoring='roc_auc')
arr

/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


array([ 0.83292867,  0.82668376,  0.80499068,  0.77795394,  0.8108635 ])

In [81]:
test = count_vectorizer.transform(tst[0])

In [82]:
algo.fit(X, y)
target = algo.predict_proba(test)

In [63]:
trained = {j:i for i, j in enumerate(data[0])}
known = set(data[0])
for i, w in enumerate(tst[0]):
    if w in known:
        target[i] = data[1][trained[w]]

In [83]:
answer = pd.read_csv('linear_ans_example.txt')
answer['Answer'] = target*1.
answer.to_csv("submission.tsv", sep=',', index=False)

In [15]:
model = RandomForestClassifier(n_estimators=1000, random_state=43)
score = model_selection.cross_val_score(model, X, y, scoring = 'roc_auc', cv = 3)
score

array([ 0.82308964,  0.76101508,  0.79930983])

In [3]:
#data[0] = data[0].map(lambda x: x.lower() if x.isupper() else x)
data['is_first_upper'] = data[0].map(lambda x: (x[0].isupper() and not x.isupper())*1.)
data[0] = data[0].map(lambda x: x.lower())

In [195]:
data.head()

,0,1,is_first_upper
0,аалтонен,1,1.0
1,аар,0,1.0
2,аарон,0,1.0
3,аарон,0,0.0
4,аарона,0,1.0


In [4]:
for i in np.arange(2, 6):
    data['last_'+str(i)+'chars'] = data[0].map(lambda x: x[-i:] if len(x) > i else x)

In [4]:
data['counter'] = data[0].map(lambda x: collections.Counter(x))
for i in np.arange(32):
    letter = chr(i+ord('а'))
    data[letter] = data['counter'].map(lambda x: x[letter])
data = data.drop('counter', axis=1)

In [6]:
data_true = data.select(lambda x: data[1][x] == 1)
data_false = data.select(lambda x: data[1][x] == 0)

In [6]:
encoder1 = LabelEncoder()#sparse=False,dtype=bool, categorical_features=np.array(set(data[['last_'+str(i)+'chars' for i in np.arange(2, 4)]]), dtype=str))
encoder1.fit(data[['last_'+str(i)+'chars' for i in np.arange(2, 5)]].as_matrix().reshape(3*len(data)))
for i in np.arange(2, 5):
    data['last_'+str(i)+'chars'] = encoder1.transform(data['last_'+str(i)+'chars'])
data.head()

,0,1,is_first_upper,last_2chars,last_3chars,last_4chars,last_5chars,а,б,в,...,ц,ч,ш,щ,ъ,ы,ь,э,ю,я
0,аалтонен,1,1.0,5245,11644,13236,тонен,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,аар,0,1.0,1128,100,100,аар,2,0,0,...,0,0,0,0,0,0,0,0,0,0
2,аарон,0,1.0,13206,15592,1233,аарон,2,0,0,...,0,0,0,0,0,0,0,0,0,0
3,аарон,0,0.0,13206,15592,1233,аарон,2,0,0,...,0,0,0,0,0,0,0,0,0,0
4,аарона,0,1.0,11341,13207,15593,арона,3,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
encoder = OneHotEncoder(sparse=False)#,dtype=bool, categorical_features=np.array(set(data[['last_'+str(i)+'chars' for i in np.arange(2, 4)]])))
new_features = encoder.fit_transform(data[['last_'+str(i)+'chars' for i in np.arange(2, 5)]])
new_features

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [8]:
names = [encoder1.inverse_transform(i) for i in set(data[['last_'+str(i)+'chars' for i in np.arange(2, 5)]].as_matrix().reshape(len(data)*3))]
oldata = data.copy()

In [11]:
new_features = new_features.transpose()

In [12]:
new_features1 = pd.DataFrame(new_features)
new_features1.columns = encoder.active_features_

In [ ]:
endings1 = set(data_true[['last_'+str(i)+'chars' for i in np.arange(2, 4)]].as_matrix().reshape(2*len(data_true)))
endings0 = set(data_false[['last_'+str(i)+'chars' for i in np.arange(2, 4)]].as_matrix().reshape(2*len(data_false)))

In [14]:
for i in np.arange(32):
    letter = chr(i+ord('а'))
    new_features1[letter] = data[letter]
new_features1['is_first_upper'] = data['is_first_upper']

In [15]:
new_features1.to_csv("some_features.tsv", sep=',')

In [13]:
data[0] = data[0].map(prep)